<a href="https://colab.research.google.com/github/leideng/AI-primer/blob/main/pytorch/tensor_slice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is a notebook about how to efficiently slide a pytorch tensor in different manners

## Python's slicing

## Boolean index

## Index

In [11]:
# Python's slicing
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
x = torch.randint(low=-5, high=5, size=(5,8), device=device)
print(f"x={x}")

x1 = x[1,:]
print(f"row 1: x1=x[1,:]={x1}")
print(f"x1.shape={x1.shape}")

y1 = x[:,1]
print(f"column 1: y1=y[:,1]={y1}")
print(f"y1.shape={y1.shape}")

x2_3 = x[2,3]
x2_35 = x[2,3:5]
print(f"x2_3=x[2,3]={x2_3}")
print(f"x2_3.shape={x2_3.shape}")
print(f"x2_35=x[2,3:5]={x2_35}")
print(f"x2_35.shape={x2_35.shape}")


x=tensor([[-5,  3, -5, -2,  3, -4,  3,  1],
        [-3,  3, -3,  4, -2,  1,  2, -4],
        [-3,  0,  0,  1,  0, -2, -4, -3],
        [ 0,  2, -3,  0,  1, -5,  3,  4],
        [-4, -1,  1, -1, -5,  3,  3,  3]])
row 1: x1=x[1,:]=tensor([-3,  3, -3,  4, -2,  1,  2, -4])
x1.shape=torch.Size([8])
column 1: y1=y[:,1]=tensor([ 3,  3,  0,  2, -1])
y1.shape=torch.Size([5])
x2_3=x[2,3]=1
x2_3.shape=torch.Size([])
x2_35=x[2,3:5]=tensor([1, 0])
x2_35.shape=torch.Size([2])
